# Regression Exercise 

California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area. 

The task is to aproximate the median house value of each block from the values of the rest of the variables. 

 It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.
 

The Features:
 
* housingMedianAge: continuous. 
* totalRooms: continuous. 
* totalBedrooms: continuous. 
* population: continuous. 
* households: continuous. 
* medianIncome: continuous. 
* medianHouseValue: continuous. 

## The Data

** Import the cal_housing_clean.csv file with pandas. Separate it into a training (70%) and testing set(30%).**

In [41]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [42]:
my_imported_data = pd.read_csv('/Users/mohamedali/Desktop/Python-DS-ML/Tensorflow/Tensorflow-Bootcamp-master/02-TensorFlow-Basics/cal_housing_clean.csv')

In [43]:
# creating dataframe for later use
my_df = pd.DataFrame(my_imported_data)

In [44]:
y_data = my_imported_data['medianHouseValue']

In [46]:
x_data = my_imported_data.drop('medianHouseValue',axis = 1)

In [47]:
 x_train, x_eval, y_train, y_eval = train_test_split(x_data, y_data, test_size= 0.3, random_state = 101)

In [48]:
my_imported_data.head()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [49]:
my_df.describe()

,housingMedianAge,totalRooms,totalBedrooms,population,households,medianIncome,medianHouseValue
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,28.639486,2635.763081,537.898014,1425.476744,499.539680,3.870671,206855.816909
std,12.585558,2181.615252,421.247906,1132.462122,382.329753,1.899822,115395.615874
min,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,18.000000,1447.750000,295.000000,787.000000,280.000000,2.563400,119600.000000
50%,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


### Scale the Feature Data

** Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.**

In [17]:
from sklearn.preprocessing import MinMaxScaler

In [50]:
scaler = MinMaxScaler()

In [52]:
scaler.fit(x_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [55]:
my_train_scale = scaler.transform(x_train)

In [69]:
my_eval_scale = scaler.transform(x_eval)

In [70]:
scaled_x_train = pd.DataFrame(data = my_train_scale, index=x_train.index, columns= x_train.columns)

In [71]:
scaled_x_eval = pd.DataFrame(data = my_eval_scale, index=x_eval.index, columns= x_eval.columns)

### Create Feature Columns

** Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns. **

In [30]:
my_imported_data.columns

Index(['housingMedianAge', 'totalRooms', 'totalBedrooms', 'population',
       'households', 'medianIncome', 'medianHouseValue'],
      dtype='object')

In [80]:
house_median = tf.feature_column.numeric_column('housingMedianAge')
total_rooms = tf.feature_column.numeric_column('totalRooms')
total_bed_rooms = tf.feature_column.numeric_column('totalBedrooms')
my_population = tf.feature_column.numeric_column('population')
my_households = tf.feature_column.numeric_column('households')
median_income = tf.feature_column.numeric_column('medianIncome')


In [81]:
feat_cols = [house_median, total_bed_rooms, total_rooms, my_population, my_households,
                median_income]

** Create the input function for the estimator object. (play around with batch_size and num_epochs)**

In [82]:
# input function
input_func = tf.estimator.inputs.pandas_input_fn(x = x_train, y = y_train,  batch_size= 10, num_epochs= 1000, shuffle=True)

** Create the estimator model. Use a DNNRegressor. Play around with the hidden units! **

In [83]:
# create dnn model
my_dnn = tf.estimator.DNNRegressor(hidden_units= [6,6,6] , feature_columns = feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/qy/8gpp2k295j980h0fyf_7b4h80000gp/T/tmpol_izu_h', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}


In [92]:
# train model
my_dnn.train(input_fn=input_func, steps = 20000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /var/folders/qy/8gpp2k295j980h0fyf_7b4h80000gp/T/tmpol_izu_h/model.ckpt-1000
INFO:tensorflow:Saving checkpoints for 1001 into /var/folders/qy/8gpp2k295j980h0fyf_7b4h80000gp/T/tmpol_izu_h/model.ckpt.
INFO:tensorflow:loss = 150803660000.0, step = 1001
INFO:tensorflow:global_step/sec: 294.212
INFO:tensorflow:loss = 67787956000.0, step = 1101 (0.342 sec)
INFO:tensorflow:global_step/sec: 358.115
INFO:tensorflow:loss = 202983100000.0, step = 1201 (0.279 sec)
INFO:tensorflow:global_step/sec: 425.907
INFO:tensorflow:loss = 146737020000.0, step = 1301 (0.235 sec)
INFO:tensorflow:global_step/sec: 387.944
INFO:tensorflow:loss = 70114860000.0, step = 1401 (0.260 sec)
INFO:tensorflow:global_step/sec: 433.9
INFO:tensorflow:loss = 257227200000.0, step = 1501 (0.232 sec)
INFO:tensorflow:global_step/sec: 428.054
INFO:tensorflow:loss = 519308940000.0, step = 1601 (0.234 sec)
INFO:tensorflow:global_step/sec: 345.988
INF

INFO:tensorflow:global_step/sec: 377.872
INFO:tensorflow:loss = 253532320000.0, step = 8701 (0.267 sec)
INFO:tensorflow:global_step/sec: 449.592
INFO:tensorflow:loss = 372906850000.0, step = 8801 (0.220 sec)
INFO:tensorflow:global_step/sec: 415.619
INFO:tensorflow:loss = 164851420000.0, step = 8901 (0.239 sec)
INFO:tensorflow:global_step/sec: 467.034
INFO:tensorflow:loss = 184654710000.0, step = 9001 (0.216 sec)
INFO:tensorflow:global_step/sec: 362.478
INFO:tensorflow:loss = 103009050000.0, step = 9101 (0.273 sec)
INFO:tensorflow:global_step/sec: 426.043
INFO:tensorflow:loss = 494981400000.0, step = 9201 (0.238 sec)
INFO:tensorflow:global_step/sec: 412.993
INFO:tensorflow:loss = 113859710000.0, step = 9301 (0.245 sec)
INFO:tensorflow:global_step/sec: 377.454
INFO:tensorflow:loss = 114980390000.0, step = 9401 (0.257 sec)
INFO:tensorflow:global_step/sec: 399.949
INFO:tensorflow:loss = 129456420000.0, step = 9501 (0.259 sec)
INFO:tensorflow:global_step/sec: 442.935
INFO:tensorflow:loss = 

INFO:tensorflow:global_step/sec: 350.23
INFO:tensorflow:loss = 50191950000.0, step = 16601 (0.280 sec)
INFO:tensorflow:global_step/sec: 316.527
INFO:tensorflow:loss = 65785140000.0, step = 16701 (0.324 sec)
INFO:tensorflow:global_step/sec: 270.317
INFO:tensorflow:loss = 157367420000.0, step = 16801 (0.370 sec)
INFO:tensorflow:global_step/sec: 201.406
INFO:tensorflow:loss = 82999850000.0, step = 16901 (0.492 sec)
INFO:tensorflow:global_step/sec: 404.294
INFO:tensorflow:loss = 100871470000.0, step = 17001 (0.247 sec)
INFO:tensorflow:global_step/sec: 238.488
INFO:tensorflow:loss = 47599300000.0, step = 17101 (0.422 sec)
INFO:tensorflow:global_step/sec: 147.588
INFO:tensorflow:loss = 101085340000.0, step = 17201 (0.676 sec)
INFO:tensorflow:global_step/sec: 244.353
INFO:tensorflow:loss = 29961790000.0, step = 17301 (0.406 sec)
INFO:tensorflow:global_step/sec: 254.496
INFO:tensorflow:loss = 52669338000.0, step = 17401 (0.392 sec)
INFO:tensorflow:global_step/sec: 107.224
INFO:tensorflow:loss 

##### ** Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) **

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\Marcial\AppData\Local\Temp\tmp244d0d44\model.ckpt.
INFO:tensorflow:step = 1, loss = 5.68896e+11
INFO:tensorflow:global_step/sec: 581.528
INFO:tensorflow:step = 101, loss = 5.52951e+11 (0.172 sec)
INFO:tensorflow:global_step/sec: 611.865
INFO:tensorflow:step = 201, loss = 5.91057e+11 (0.163 sec)
INFO:tensorflow:global_step/sec: 619.467
INFO:tensorflow:step = 301, loss = 5.48503e+11 (0.162 sec)
INFO:tensorflow:global_step/sec: 683.11
INFO:tensorflow:step = 401, loss = 1.96651e+11 (0.146 sec)
INFO:tensorflow:global_step/sec: 671.59
INFO:tensorflow:step = 501, loss = 3.18206e+11 (0.149 sec)
INFO:tensorflow:global_step/sec: 680.779
INFO:tensorflow:step = 601, loss = 7.34722e+11 (0.147 sec)
INFO:tensorflow:global_step/sec: 683.111
INFO:tensorflow:step = 701, loss = 6.52219e+11 (0.146 sec)
INFO:tensorflow:global_step/sec: 641.376
INFO:tensorflow:step = 801, loss = 1.67628e+11 (0.155 sec)
INFO:te

INFO:tensorflow:global_step/sec: 619.468
INFO:tensorflow:step = 8101, loss = 6.23952e+10 (0.161 sec)
INFO:tensorflow:global_step/sec: 676.164
INFO:tensorflow:step = 8201, loss = 1.45558e+11 (0.148 sec)
INFO:tensorflow:global_step/sec: 667.117
INFO:tensorflow:step = 8301, loss = 5.34845e+10 (0.149 sec)
INFO:tensorflow:global_step/sec: 687.821
INFO:tensorflow:step = 8401, loss = 8.33874e+10 (0.145 sec)
INFO:tensorflow:global_step/sec: 673.836
INFO:tensorflow:step = 8501, loss = 1.16478e+11 (0.149 sec)
INFO:tensorflow:global_step/sec: 683.109
INFO:tensorflow:step = 8601, loss = 1.72198e+11 (0.146 sec)
INFO:tensorflow:global_step/sec: 676.163
INFO:tensorflow:step = 8701, loss = 1.35063e+11 (0.148 sec)
INFO:tensorflow:global_step/sec: 674.01
INFO:tensorflow:step = 8801, loss = 1.63489e+11 (0.148 sec)
INFO:tensorflow:global_step/sec: 639.32
INFO:tensorflow:step = 8901, loss = 9.24273e+10 (0.156 sec)
INFO:tensorflow:global_step/sec: 637.281
INFO:tensorflow:step = 9001, loss = 1.02409e+11 (0.1

INFO:tensorflow:global_step/sec: 660.481
INFO:tensorflow:step = 16201, loss = 3.05991e+10 (0.151 sec)
INFO:tensorflow:global_step/sec: 685.456
INFO:tensorflow:step = 16301, loss = 1.88008e+11 (0.146 sec)
INFO:tensorflow:global_step/sec: 683.115
INFO:tensorflow:step = 16401, loss = 1.83709e+11 (0.146 sec)
INFO:tensorflow:global_step/sec: 676.163
INFO:tensorflow:step = 16501, loss = 1.30615e+11 (0.148 sec)
INFO:tensorflow:global_step/sec: 676.163
INFO:tensorflow:step = 16601, loss = 1.21363e+11 (0.148 sec)
INFO:tensorflow:global_step/sec: 662.686
INFO:tensorflow:step = 16701, loss = 5.79005e+10 (0.150 sec)
INFO:tensorflow:global_step/sec: 676.164
INFO:tensorflow:step = 16801, loss = 2.05555e+11 (0.148 sec)
INFO:tensorflow:global_step/sec: 637.28
INFO:tensorflow:step = 16901, loss = 1.13958e+11 (0.157 sec)
INFO:tensorflow:global_step/sec: 656.145
INFO:tensorflow:step = 17001, loss = 1.56865e+11 (0.152 sec)
INFO:tensorflow:global_step/sec: 678.463
INFO:tensorflow:step = 17101, loss = 1.035

INFO:tensorflow:global_step/sec: 651.856
INFO:tensorflow:step = 24301, loss = 7.52043e+10 (0.153 sec)
INFO:tensorflow:global_step/sec: 676.164
INFO:tensorflow:step = 24401, loss = 1.08424e+11 (0.147 sec)
INFO:tensorflow:global_step/sec: 669.356
INFO:tensorflow:step = 24501, loss = 9.23162e+10 (0.150 sec)
INFO:tensorflow:global_step/sec: 680.779
INFO:tensorflow:step = 24601, loss = 5.40436e+10 (0.147 sec)
INFO:tensorflow:global_step/sec: 683.097
INFO:tensorflow:step = 24701, loss = 8.55543e+10 (0.146 sec)
INFO:tensorflow:global_step/sec: 653.971
INFO:tensorflow:step = 24801, loss = 1.211e+11 (0.153 sec)
INFO:tensorflow:global_step/sec: 629.241
INFO:tensorflow:step = 24901, loss = 6.22166e+10 (0.159 sec)
INFO:tensorflow:Saving checkpoints for 25000 into C:\Users\Marcial\AppData\Local\Temp\tmp244d0d44\model.ckpt.
INFO:tensorflow:Loss for final step: 8.48849e+10.


** Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data. **

In [101]:
pred_input = tf.estimator.inputs.pandas_input_fn(x=x_data, batch_size= 10,
                                                 num_epochs= 1, shuffle=False)

In [102]:
prediction = my_dnn.predict(pred_input)

In [103]:
my_new_prediction  = list(prediction)

INFO:tensorflow:Restoring parameters from /var/folders/qy/8gpp2k295j980h0fyf_7b4h80000gp/T/tmpol_izu_h/model.ckpt-21000


In [104]:
my_new_prediction

[{'predictions': array([238497.44], dtype=float32)},
 {'predictions': array([377246.16], dtype=float32)},
 {'predictions': array([297930.66], dtype=float32)},
 {'predictions': array([280152.06], dtype=float32)},
 {'predictions': array([292072.], dtype=float32)},
 {'predictions': array([265330.88], dtype=float32)},
 {'predictions': array([317683.34], dtype=float32)},
 {'predictions': array([344714.16], dtype=float32)},
 {'predictions': array([261473.02], dtype=float32)},
 {'predictions': array([345941.97], dtype=float32)},
 {'predictions': array([300857.56], dtype=float32)},
 {'predictions': array([345696.53], dtype=float32)},
 {'predictions': array([306620.4], dtype=float32)},
 {'predictions': array([252806.05], dtype=float32)},
 {'predictions': array([312943.78], dtype=float32)},
 {'predictions': array([241841.53], dtype=float32)},
 {'predictions': array([292060.34], dtype=float32)},
 {'predictions': array([264724.03], dtype=float32)},
 {'predictions': array([283863.72], dtype=float32

** Calculate the RMSE. You should be able to get around 100,000 RMSE (remember that this is in the same units as the label.) Do this manually or use [sklearn.metrics](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) **

In [106]:
final_predictions = []

for pred in my_new_prediction:
    final_predictions.append(pred['predictions'])

In [107]:
final_predictions

[array([238497.44], dtype=float32),
 array([377246.16], dtype=float32),
 array([297930.66], dtype=float32),
 array([280152.06], dtype=float32),
 array([292072.], dtype=float32),
 array([265330.88], dtype=float32),
 array([317683.34], dtype=float32),
 array([344714.16], dtype=float32),
 array([261473.02], dtype=float32),
 array([345941.97], dtype=float32),
 array([300857.56], dtype=float32),
 array([345696.53], dtype=float32),
 array([306620.4], dtype=float32),
 array([252806.05], dtype=float32),
 array([312943.78], dtype=float32),
 array([241841.53], dtype=float32),
 array([292060.34], dtype=float32),
 array([264724.03], dtype=float32),
 array([283863.72], dtype=float32),
 array([271959.3], dtype=float32),
 array([193795.48], dtype=float32),
 array([223072.47], dtype=float32),
 array([301787.47], dtype=float32),
 array([271034.97], dtype=float32),
 array([294525.25], dtype=float32),
 array([196339.69], dtype=float32),
 array([245643.17], dtype=float32),
 array([264895.84], dtype=float3

In [108]:
from sklearn.metrics import mean_squared_error

In [118]:
mean_squared_error(y_eval ,final_predictions) ** 0.5

# print("this is the y-eval size" , y_eval.shape)
# print("this is the final_predictions shape" , len(final_predictions))

ValueError: Found input variables with inconsistent numbers of samples: [6192, 20640]

97921.93181985477

# Great Job!